# Collink | the video call Add-on 

    Have you ever been in a Video Call within your company or university and came across people u don't know? 

    I've always been curious about former colleagues or students in my projects or courses. 

    With the collink Add-on you should be able to identify the LinkedIn profile page of your colleague by scanning his/her face. 
    
<img src="funny.gif" width="200" align="center">

                     okay jokes aside. the algorithm is a low level GUI version of the ideal idea. 

<img src="zoom.gif" width="750" align="center">

                                                Lets get into it now.

In [ ]:
import face_recognition
import numpy as np
import cv2

from io import BytesIO
from PIL import Image

import pickle
import argparse
import imutils 
import requests
import time 
import os
import random
import fnmatch
from matplotlib import pyplot as plt
import warnings
import pandas as pd
import uuid
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import difflib
import json 
from pprint import pprint
import webbrowser

import config
from linkedin_api import Linkedin
import sys
import urllib.request
from bs4 import BeautifulSoup as bs
from google_images_search import GoogleImagesSearch

from termcolor import colored as col
from textblob import TextBlob


import urllib

ended=False ##. variable to break th emain loop 
api=Linkedin("elisabeth.oeljeklaus@web.de",config.password)

with open("get_known_people_encodings.pickle", "rb") as f:
    known_face_encodings,known_face_names =pickle.load(f)
    
    
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

video_capture = cv2.VideoCapture(0)
cv2.CAP_PROP_BUFFERSIZE 

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    
    #rectangle with description
    cv2.rectangle(frame, (0, 0), (350,50), (127, 120, 121), cv2.FILLED)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(frame, "for results press L", (0, 35), font, 1.0, (255, 255, 255), 1)
    
    

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
    
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        if(len(face_encodings)>0):
            matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])


            # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
                first_match_index = matches.index(True)
                name = str(known_face_names[first_match_index])
                indx= name.find("_")
                l_name=name[indx+1:]
                f_name=name[:indx]


            else: 
                x="Face does not exist in database."
     
    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left) in face_locations:
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (98, 229, 111), 2)
        
        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (98, 229, 111), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, "searching . . .", (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    cv2.startWindowThread()
    cv2.imshow('Video', frame) # important to put below all cv2. instructions
    cv2.setWindowProperty('Video', cv2.WND_PROP_VISIBLE, 1)

    cv2.setWindowProperty('Video', cv2.WND_PROP_TOPMOST, 1)

    # Hit 'l' on the keyboard to get LinkedIn profile!
    if cv2.waitKey(1) & 0xFF == ord('l'):
        cv2.destroyAllWindows()
        cv2.waitKey(5)
        video_capture.release()
        person_search=api.search_people(keyword_first_name=f_name, keyword_last_name=l_name)
        res_url= person_search[0]["urn_id"]
        res_name= person_search[0]["name"]
        job=person_search[0]["jobtitle"]
        loc=person_search[0]["location"]
        info=api.get_profile(res_url)
        industry=info["industryName"]
        company = info["experience"][0]["companyName"]
        job_=info["experience"][0]["title"]
        
        print("------------------ Here are the results -------------------------")
        print()
        print("The person could be: ",col(res_name,"green",attrs=["bold"]))
        print("working as a ",col(job,"blue",attrs=["bold"]),"in ",col(loc,"blue",attrs=["bold"]))
        print("http://www.linkedin.com/in/"+res_url)
        
   
        #open linkedin tab in browser
        webbrowser.open("http://www.linkedin.com/in/"+res_url)
        
        #open new window with results
        path = "pics/GUI_interface.jpeg"
        image = cv2.imread(path)
        height, width, channels = image.shape
        start_point = (0,0)
        end_point = (width, height)
        color = (255,255,255)
        thickness = 5

        image = cv2.rectangle(image, start_point, end_point, color, thickness)
        
        while True: 
            #cv2.setWindowProperty("fullscreen", cv.WND_PROP_FULLSCREEN,cv.WINDOW_FULLSCREEN)
            cv2.imshow('Result',image)
            cv2.setWindowProperty('Result', cv2.WND_PROP_TOPMOST, 1)

            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(image, ("Result"), (680, 238), font, 2.0, (255, 255, 255), 1)
            cv2.putText(image, ("Name: "+res_name), (311, 318), font, 1.0, (255, 255, 255), 1)
            cv2.putText(image, ("Industry: "+industry), (311, 357), font, 1.0, (255, 255, 255), 1)
            cv2.putText(image, ("Job: "+job_), (311, 396), font, 1.0, (255, 255, 255), 1)
            cv2.putText(image, ("Company: "+company), (311, 435), font, 1.0, (255, 255, 255), 1)
            cv2.putText(image, ("Location: "+loc), (311, 474), font, 1.0, (255, 255, 255), 1)
            cv2.putText(image, ("to close press 'q'"), (734, 577), font, 1.0, (255, 255, 255), 1)
            
            if cv2.waitKey(1) & 0xFF == ord("q"):
                ended=True
                print("test")
                #cv2.destroyWindow("Result")
                #cv2.destroyAllWindows()
                #cv2.waitKey(1)
                break
       
    elif cv2.waitKey(1) & 0xFF == ord("q"):
        break

    if ended: 
        print("finished the main loop")
        break    

print("program finished...")
cv2.destroyAllWindows()
cv2.waitKey(1)
#video_capture.release()
#raise KeyboardInterrupt
#assert(False)
#!jupyter notebook stop
import os
os._exit(0)

------------------ Here are the results -------------------------

The person could be:  Joyce Bride
working as a  Program Associate at Ironhack in  Berlin, Berlin, Germany
http://www.linkedin.com/in/ACoAACI02BgBjcFGyDcj7o2zXkJ5U3jaj0nJ1pM


# The current workflow 


1. Gather a database from known faces 
        - Do it manually (download jpeg/jpg and name the file with name)
        
2. Run the encode_people_faces function 
        - Function and this sub workflow is stored in extra jupyter notebook
        - Save the functions output with pickle 
        
3. Run the main algorithm 
       - Import the alle necessarly face_recognition, selenium, pickle, cv2
       - comparing live face with known faces in data base 
       - find match 
       - open LinkedIn profile by using LinkedIn API 
       - open results window with open cv 
